# Models


## Installs


In [ ]:
pip install ultralytics

In [5]:
import os
import cv2
import numpy as np


def get_yolo_preds():
    """
    Run YOLO predictions, save cropped persons, and save the whole image with bounding boxes.

    Args:
        net: YOLO model loaded with cv2.dnn.readNetFromDarknet.
        input_img_path (str): Path to the input image.
        output_img_path (str): Path to save the output image with bounding boxes.
        confidence_threshold (float): Minimum confidence for predictions.
        overlapping_threshold (float): Threshold for non-maxima suppression.
        labels (list): List of class labels.
        show_display (bool): Whether to display the image with bounding boxes.
    """
            # Load COCO labels
    labels_path = "./DARKNET/coco.txt"
    try:
        with open(labels_path, "r", encoding="utf-8") as f:
            labels = f.read().strip().split("\n")
    except FileNotFoundError:
        print(f"Error: {labels_path} not found. Please check the file path.")
        exit()

    # Define YOLO paths and parameters
    yolo_cfg = "./DARKNET/model_data/yolov3.cfg"
    yolo_weights = "./DARKNET/model_data/yolov3.weights"
    input_img_path = "./DARKNET/test3.jpg"
    output_img_path = "./DARKNET/output"
    confidence_threshold = 0.5
    overlapping_threshold = 0.5

    # Initialize YOLO model
    net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)
    # if cuda: 
    #     net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    #     net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    # Ensure output directory exists if the output_img_path is provided
    if output_img_path:
        output_dir = os.path.dirname(output_img_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

    # Generate random colors for each class
    np.random.seed(123)
    colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

    # Get the output layer names of the YOLO network
    ln = net.getLayerNames()
    ln = [ln[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

    # Read the input image
    image = cv2.imread(input_img_path)
    if image is None:
        print(f"Error: Unable to load image {input_img_path}")
        return

    (H, W) = image.shape[:2]

    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    # Initialize lists for detections
    boxes, confidences, classIDs = [], [], []

    # Loop through each output layer
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Only keep detections for "person" (class ID 0)
            if confidence > confidence_threshold and classID == 0:
                # Scale bounding boxes back to image dimensions
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # Perform non-maxima suppression to reduce overlapping boxes
    bboxes = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, overlapping_threshold)
    image2 = image.copy()
    # Collect cropped persons and draw bounding boxes on the image
    cropped_persons = []
    if len(bboxes) > 0:
        for i in bboxes.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # Draw bounding box on the original image
            color = [int(c) for c in colors[classIDs[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = f"{labels[classIDs[i]]}: {confidences[i]:.2f}"
            cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Crop the person from the image (no bounding box)
            cropped_person = image2[y:y + h, x:x + w]
            cropped_persons.append(cropped_person)

            # Save the cropped person image
            cropped_filename = f"./DARKNET/output/person_{i + 1}_confidence_{confidences[i]:.2f}.jpg"
            cv2.imwrite(cropped_filename, cropped_person)
            print(f"Cropped Person Saved: {cropped_filename}")

    # Save the full image with bounding boxes
    if output_img_path:
        if not output_img_path.lower().endswith(('.jpg', '.jpeg', '.png')):
            output_img_path += '.jpg'  # Add .jpg extension if none is provided
        if cv2.imwrite(output_img_path, image):
            print(f"Image with bounding boxes saved to {output_img_path}")
        else:
            print("Failed to save the image with bounding boxes.")





# Run YOLO predictions
if __name__ == '__main__':
    get_yolo_preds()



Cropped Person Saved: ./DARKNET/output/person_3_confidence_0.99.jpg
Image with bounding boxes saved to ./DARKNET/output.jpg


# Resnet50xInceptionv3 Training


In [161]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, InceptionV3  
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet 
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Flatten, Dense, Concatenate, Input, BatchNormalization, ReLU, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from torchsummary import summary  # Install with pip install torchsummary

import numpy as np
import os

# Dataset path
# dataset_path = "./cropped_images"
dataset_path = r"D:\THESIS PROJECT FOLDER\test"
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Directory not found: {dataset_path}")

# Load dataset with different image sizes
dataset1 = image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),  # Resize for ResNet50 (224x224)
    batch_size=32,
    shuffle=True
)

dataset2 = image_dataset_from_directory(
    dataset_path,
    image_size=(299, 299),  # Resize for InceptionV3 (299x299)
    batch_size=32,
    shuffle=True
)

# Extract class names from the first dataset (both datasets should have the same class names)
class_names = dataset1.class_names  # ['class1', 'class2']
num_classes = len(class_names)

# Preprocess the dataset for ResNet50
def preprocess_dataset(dataset, preprocess_fn):
    images, labels1 = [], []
    for img_batch, label_batch in dataset:
        for img, label in zip(img_batch, label_batch):
            images.append(preprocess_fn(img.numpy()))  # Apply preprocessing
            labels1.append(label.numpy())  # Add label
    return np.array(images), np.array(labels1)

# Preprocess for ResNet50 and InceptionV3
images_resnet, labels1 = preprocess_dataset(dataset1, preprocess_resnet)
images_inception, _ = preprocess_dataset(dataset2, preprocess_inception)

# Load pre-trained ResNet50 and InceptionV3 models (without top layers)
base_model_resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model_inception = InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))

# Freezing all layers of the base models
base_model_resnet.trainable = False
base_model_inception.trainable = False
# Unfreeze some layers for fine-tuning (optional)
for layer in base_model_resnet.layers[-94:]:
    layer.trainable = True

for layer in base_model_inception.layers[-50:]:
    layer.trainable = True

# Create feature extractor models
feature_extractor_resnet = Model(inputs=base_model_resnet.input, outputs=base_model_resnet.output)
feature_extractor_inception = Model(inputs=base_model_inception.input, outputs=base_model_inception.output)

# Extract feature maps
print("Extracting features from ResNet50...")
features_resnet = feature_extractor_resnet.predict(images_resnet)
print("ResNet50 Feature Map Shape:", features_resnet.shape)

print("Extracting features from InceptionV3...")
features_inception = feature_extractor_inception.predict(images_inception)
print("InceptionV3 Feature Map Shape:", features_inception.shape)

# Apply Global Average Pooling (GAP) to both ResNet and Inception features
gap_resnet = GlobalAveragePooling2D()(features_resnet)
gap_inception = GlobalAveragePooling2D()(features_inception)

# Concatenate the GAP features
print("Concatenating feature maps...")
fused_features = np.concatenate([gap_resnet, gap_inception], axis=1)
print("Fused Feature Shape:", fused_features.shape)

# Apply Batch Normalization
bn_layer = BatchNormalization()(fused_features)

# Apply ReLU activation
relu_layer = ReLU()(bn_layer)

# Apply Dropout for regularization
dropout_layer = Dropout(0.5)(relu_layer)  # Dropout rate can be adjusted

# Build the classifier model
print("Building the Sigmoid classifier model...")

# Define the input layer for the classifier
input_layer = Input(shape=(dropout_layer.shape[1],))

# Output layer with sigmoid activation for binary classification
output_layer = Dense(1, activation='sigmoid')(input_layer)

# Final model
classifier_model1 = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
classifier_model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Found 222 files belonging to 2 classes.
Found 222 files belonging to 2 classes.
Extracting features from ResNet50...
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step
ResNet50 Feature Map Shape: (222, 7, 7, 2048)
Extracting features from InceptionV3...
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step
InceptionV3 Feature Map Shape: (222, 8, 8, 2048)
Concatenating feature maps...
Fused Feature Shape: (222, 4096)
Building the Sigmoid classifier model...


In [3]:
import tensorflow as tf

print(tf.__version__)


2.18.0


In [4]:
# Train-test split
X_train1, X_test1, y_train1, y_test1 = train_test_split(fused_features, labels1, test_size=0.2, random_state=42)
print("Training the classifier after unfreezing some layers...")

history_finetune = classifier_model1.fit(
    X_train1, y_train1,
    validation_split=0.2,
    epochs=5,
    batch_size=32,
    verbose=1
)

# Evaluate the classifier
print("Evaluating the classifier...")
y_pred1 = classifier_model1.predict(X_test1)
y_pred_classes = (y_pred1 > 0.5).astype(int)  # Convert probabilities to binary predictions

print("Classification Report:")
print(classification_report(y_test1, y_pred_classes, target_names=class_names))
print("Accuracy Score:", accuracy_score(y_test1, y_pred_classes))
# Save the model
# model_save_path = 'Saved models/resnetxinception.keras'  # Specify the file path and name
# classifier_model1.save(model_save_path)
# print(f"Model saved to {model_save_path}")




NameError: name 'train_test_split' is not defined

In [ ]:
import tensorflow as tf
import numpy as np

#KATONG DERETSO WITHOUT LOADING MODEL
# # Predict on a new image
# def predict_image(img_path):
#     img_resnet = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
#     img_inception = img_resnet.copy()

#     img_array_resnet = preprocess_resnet(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img_resnet), axis=0))
#     img_array_inception = preprocess_inception(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img_inception), axis=0))

#     feature_maps_resnet = feature_extractor_resnet.predict(img_array_resnet)
#     feature_maps_inception = feature_extractor_inception.predict(img_array_inception)

#     # Apply GAP to the feature maps
#     gap_resnet = np.mean(feature_maps_resnet, axis=(1, 2))  # Global Average Pooling
#     gap_inception = np.mean(feature_maps_inception, axis=(1, 2))  # Global Average Pooling

#     # Concatenate the GAP features
#     fused_features = np.concatenate([gap_resnet, gap_inception]).reshape(1, -1)
    
#     # Make prediction
#     # prediction = classifier_model.predict(fused_features)
#     # return class_names[int(prediction[0] > 0.5)]  # Return class based on predicted probability
#     prediction = classifier_model1.predict(fused_features)
#     # Extract the scalar value properly
#     predicted_value = prediction[0].item()  # Ensure a scalar is extracted
#     return class_names[int(predicted_value > 0.5)]

# # Test prediction
# new_img_path = './DARKNET/test.jpg'  # Replace with the path to a new image
# predicted_class = predict_image(new_img_path)
# print(f"Predicted Class for {new_img_path}: {predicted_class}")

# Load the model
model_save_path = 'Saved models/resnetxinception.keras'  # Path to your saved model
classifier_model1 = tf.keras.models.load_model(model_save_path)

# Load the feature extractors (ResNet and Inception) from the saved model
base_model_resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model_inception = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))

# Freezing all layers of the base models (if not already done)
base_model_resnet.trainable = False
base_model_inception.trainable = False

# Create feature extractor models
feature_extractor_resnet = tf.keras.Model(inputs=base_model_resnet.input, outputs=base_model_resnet.output)
feature_extractor_inception = tf.keras.Model(inputs=base_model_inception.input, outputs=base_model_inception.output)

# Predict on a new image
def predict_image(img_path):
    # Load the image for ResNet50 and resize it
    img_resnet = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    # Load the image for InceptionV3 and resize it accordingly
    img_inception = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299))

    # Preprocess images
    img_array_resnet = preprocess_resnet(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img_resnet), axis=0))
    img_array_inception = preprocess_inception(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img_inception), axis=0))

    # Extract feature maps using both ResNet50 and InceptionV3
    feature_maps_resnet = feature_extractor_resnet.predict(img_array_resnet)
    feature_maps_inception = feature_extractor_inception.predict(img_array_inception)

    # Apply Global Average Pooling (GAP) to the feature maps
    gap_resnet = np.mean(feature_maps_resnet, axis=(1, 2))  # Global Average Pooling
    gap_inception = np.mean(feature_maps_inception, axis=(1, 2))  # Global Average Pooling

    # Concatenate the GAP features
    fused_features = np.concatenate([gap_resnet, gap_inception]).reshape(1, -1)
    
    # Make prediction using the loaded classifier model
    prediction = classifier_model1.predict(fused_features)

    # Extract the scalar value properly
    predicted_value = prediction[0].item()  # Ensure a scalar is extracted
    return class_names[int(predicted_value > 0.5)]

# Test prediction
new_img_path = './DARKNET/test2.jpg'  # Replace with the path to a new image
predicted_class = predict_image(new_img_path)
print(f"Predicted Class for {new_img_path}: {predicted_class}")

classifier_model1.summary()

In [ ]:
print("ResNet50 Trainable Layers:")
for i, layer in enumerate(base_model_resnet.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

print("\nInceptionV3 Trainable Layers:")
for i, layer in enumerate(base_model_inception.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

classifier_model1.summary()
# Define the complete model by combining feature extractors and the classifier
complete_model = Model(
    inputs=[base_model_resnet.input, base_model_inception.input],  # Inputs from both models
    outputs=output_layer  # Output from the classifier
)

# Print the summary of the entire model
complete_model.summary()

##WHOLE MODEL RESNETXINCEPTION

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, InceptionV3  
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet 
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers import Flatten, Dense, Concatenate, Input, BatchNormalization, ReLU, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import os

# Dataset path
# dataset_path = r"D:\THESIS PROJECT FOLDER\test"
dataset_path = "./cropped_images"
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Directory not found: {dataset_path}")

# Load dataset with different image sizes
dataset1 = image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),  # Resize for ResNet50 (224x224)
    batch_size=32,
    shuffle=True
)

dataset2 = image_dataset_from_directory(
    dataset_path,
    image_size=(299, 299),  # Resize for InceptionV3 (299x299)
    batch_size=32,
    shuffle=True
)

# Extract class names
class_names = dataset1.class_names
num_classes = len(class_names)

# Preprocess the dataset
def preprocess_dataset(dataset, preprocess_fn):
    images, labels1 = [], []
    for img_batch, label_batch in dataset:
        for img, label in zip(img_batch, label_batch):
            images.append(preprocess_fn(img.numpy()))  # Apply preprocessing
            labels1.append(label.numpy())  # Add label
    return np.array(images), np.array(labels1)

# Preprocess for ResNet50 and InceptionV3
images_resnet, labels1 = preprocess_dataset(dataset1, preprocess_resnet)
images_inception, _ = preprocess_dataset(dataset2, preprocess_inception)

# Train-test split
X_train_resnet, X_val_resnet, y_train, y_val = train_test_split(images_resnet, labels1, test_size=0.2, random_state=42)
X_train_inception, X_val_inception, _, _ = train_test_split(images_inception, labels1, test_size=0.2, random_state=42)

# Define models
base_model_resnet = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model_inception = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(299, 299, 3))
base_model_resnet.trainable = False
base_model_inception.trainable = False


# Define inputs
input_resnet = Input(shape=(224, 224, 3))
input_inception = Input(shape=(299, 299, 3))

# Extract features
features_resnet = GlobalAveragePooling2D()(base_model_resnet(input_resnet))
features_inception = GlobalAveragePooling2D()(base_model_inception(input_inception))

# Fuse features
fused_features1 = Concatenate(axis=1)([features_resnet, features_inception])
fused_features1 = BatchNormalization()(fused_features1)
fused_features1 = ReLU()(fused_features1)
fused_features1 = Dropout(0.55)(fused_features1)

# Output layer
output_layer = Dense(1, activation='sigmoid')(fused_features1)

# Final model
classifier_model1 = Model(inputs=[input_resnet, input_inception], outputs=output_layer)
classifier_model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Found 1667 files belonging to 2 classes.
Found 1667 files belonging to 2 classes.


In [2]:

# Train the model
history = classifier_model1.fit(
    [X_train_resnet, X_train_inception], 
    y_train,
    validation_data=([X_val_resnet, X_val_inception], y_val),
    epochs=10
)

# Evaluate the model
val_loss, val_acc = classifier_model1.evaluate([X_val_resnet, X_val_inception], y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

# Predictions and metrics
y_pred = classifier_model1.predict([X_val_resnet, X_val_inception])
y_pred_bin = (y_pred > 0.5).astype(int)
print(classification_report(y_val, y_pred_bin))
print(f"Accuracy Score: {accuracy_score(y_val, y_pred_bin)}")



Epoch 1/10


E:\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_486', 'keras_tensor_487']. Received: the structure of inputs=('*', '*')
  warnings.warn(


42/42 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - accuracy: 0.6383 - loss: 0.7256 - val_accuracy: 0.8413 - val_loss: 0.3381
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.8224 - loss: 0.3976 - val_accuracy: 0.9012 - val_loss: 0.2529
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 130s 3s/step - accuracy: 0.8836 - loss: 0.2822 - val_accuracy: 0.8862 - val_loss: 0.2436
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 126s 3s/step - accuracy: 0.9010 - loss: 0.2555 - val_accuracy: 0.9042 - val_loss: 0.2262
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.9332 - loss: 0.1953 - val_accuracy: 0.9102 - val_loss: 0.2089
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.9078 - loss: 0.2125 - val_accuracy: 0.9042 - val_loss: 0.2231
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.9188 - loss: 0.2249 - val_accuracy: 0.8982 - val_loss: 0.2367
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.9285 - loss: 0.1698 - val_accuracy: 0.9072 - val_loss: 0.

In [3]:
# Unfreeze some layers for fine-tuning (optional)
for layer in base_model_resnet.layers[-94:]:
    layer.trainable = True

for layer in base_model_inception.layers[-50:]:
    layer.trainable = True
    # Train the model
    
print("SECOND TRAINING")

history = classifier_model1.fit(
    [X_train_resnet, X_train_inception], 
    y_train,
    validation_data=([X_val_resnet, X_val_inception], y_val),
    epochs=10
)

# Evaluate the model
val_loss, val_acc = classifier_model1.evaluate([X_val_resnet, X_val_inception], y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

# Predictions and metrics
y_pred = classifier_model1.predict([X_val_resnet, X_val_inception])
y_pred_bin = (y_pred > 0.5).astype(int)
print(classification_report(y_val, y_pred_bin))
print(f"Accuracy Score: {accuracy_score(y_val, y_pred_bin)}")

SECOND TRAINING
Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 125s 3s/step - accuracy: 0.9368 - loss: 0.1423 - val_accuracy: 0.9012 - val_loss: 0.2234
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 124s 3s/step - accuracy: 0.9562 - loss: 0.1147 - val_accuracy: 0.8982 - val_loss: 0.2264
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 125s 3s/step - accuracy: 0.9480 - loss: 0.1276 - val_accuracy: 0.9072 - val_loss: 0.2154
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 125s 3s/step - accuracy: 0.9476 - loss: 0.1293 - val_accuracy: 0.8982 - val_loss: 0.2256
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 125s 3s/step - accuracy: 0.9467 - loss: 0.1295 - val_accuracy: 0.8922 - val_loss: 0.2209
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.9704 - loss: 0.0933 - val_accuracy: 0.8952 - val_loss: 0.2291
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 126s 3s/step - accuracy: 0.9634 - loss: 0.0892 - val_accuracy: 0.8922 - val_loss: 0.2302
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 127s 3s/step - accuracy: 0.9617 - loss: 0.1010 - val_accu

In [13]:
# Ensure training and validation inputs are properly structured
X_train = [np.array(X_train_resnet), np.array(X_train_inception)]
X_val = [np.array(X_val_resnet), np.array(X_val_inception)]

# Train the model
history = classifier_model1.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=10
)

# Evaluate the model
val_loss, val_acc = classifier_model1.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

# Make predictions
y_pred = classifier_model1.predict(X_val)
y_pred_bin = (y_pred > 0.5).astype(int)

# Print classification metrics
print(classification_report(y_val, y_pred_bin))
print(f"Accuracy Score: {accuracy_score(y_val, y_pred_bin)}")

Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 135s 3s/step - accuracy: 0.9531 - loss: 0.1178 - val_accuracy: 0.9192 - val_loss: 0.2714
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 134s 3s/step - accuracy: 0.9540 - loss: 0.1141 - val_accuracy: 0.9132 - val_loss: 0.2612
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 132s 3s/step - accuracy: 0.9409 - loss: 0.1441 - val_accuracy: 0.9281 - val_loss: 0.2626
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.9606 - loss: 0.1046 - val_accuracy: 0.9192 - val_loss: 0.2704
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 131s 3s/step - accuracy: 0.9589 - loss: 0.1195 - val_accuracy: 0.9042 - val_loss: 0.3090
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.9552 - loss: 0.1123 - val_accuracy: 0.9132 - val_loss: 0.2645
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - accuracy: 0.9604 - loss: 0.1031 - val_accuracy: 0.9222 - val_loss: 0.2825
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 133s 3s/step - accuracy: 0.9538 - loss: 0.1297 - val_accuracy: 0.8952 - v

In [4]:
# Save the model
model_save_path = 'Saved models/resnetxinceptionV5_TRAINED_ON_CROPPED.keras'  # Specify the file path and name
classifier_model1.save(model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to Saved models/resnetxinceptionV5_TRAINED_ON_CROPPED.keras


In [3]:
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet 
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception
import tensorflow as tf
import numpy as np

# Load the model (which includes both feature extractors and classifier layers)
model_save_path = 'Saved models/resnetxinceptionVVV2.keras'  # Path to your saved model
classifier_model1 = tf.keras.models.load_model(model_save_path)

# Function to predict the class of an uploaded image
def predict_image(img_path):
    # Load the image for ResNet50 and resize it
    # img_resnet = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    # # Load the image for InceptionV3 and resize it accordingly
    # img_inception = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299))
    # Ensures the image is loaded as RGB
    img_resnet = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224), color_mode='rgb')
    img_inception = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299), color_mode='rgb')


    # Preprocess images for ResNet50 and InceptionV3
    img_array_resnet = preprocess_resnet(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img_resnet), axis=0))
    img_array_inception = preprocess_inception(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img_inception), axis=0))

    # Predict the class using the loaded model (model takes preprocessed images directly)
    prediction = classifier_model1.predict([img_array_resnet, img_array_inception])

     # Extract the scalar value properly
    predicted_value = prediction[0].item()  # Ensure a scalar is extracted
    return class_names[int(predicted_value > 0.5)]
 

# Test prediction
new_img_path = './DARKNET/ZEB.jpg'  # Replace with the path to a new image
predicted_class = predict_image(new_img_path)
print(f"Predicted Class for {new_img_path}: {predicted_class}")

classifier_model1.summary()


E:\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 296 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
E:\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_layer_132', 'input_layer_133']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


NameError: name 'class_names' is not defined

# ResnetxSVM

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.utils import image_dataset_from_directory
import os

# Dataset path
dataset_path = "./cropped_images"
if not os.path.exists(dataset_path):
    print(f"Directory not found: {dataset_path}")
else:
    print(f"Directory found: {dataset_path}")
    print("Contents:", os.listdir(dataset_path))

# Load the dataset and split into training and validation sets
dataset = image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),  # ResNet50 expects input size 224x224
    batch_size=32,
    shuffle=True
)

# Extract class names for labels
class_names = dataset.class_names  # ['class1', 'class2']

# Step 2: Preprocess images and labels for feature extraction
def preprocess_dataset(dataset):
    images, labels = [], []
    for img_batch, label_batch in dataset:
        for img, label in zip(img_batch, label_batch):
            images.append(preprocess_input(img.numpy()))  # Preprocess images
            labels.append(label.numpy())  # Add corresponding label
    return np.array(images), np.array(labels)

images, labels = preprocess_dataset(dataset)

# Step 3: Load the ResNet50 model pre-trained on ImageNet (exclude top layer)
base_model = ResNet50(
    include_top=False,  # Exclude the classification layers
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# Freeze the initial layers and unfreeze the top layers
base_model.trainable = False


# Apply Global Average Pooling to the output of ResNet50 to reduce it to a 2D vector
gap_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

# Create the feature extractor model
feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=gap_layer)

# Step 4: Extract features from images (forward pass through the network)
print("Extracting features from images...")
features = feature_extractor.predict(images)
print("Feature Map Shape (Batch, Channels):", features.shape)

# Step 5: Flatten the features
features_flattened = features.reshape(features.shape[0], -1)  # Flatten (batch_size, channels)
print("Flattened Feature Shape:", features_flattened.shape)




Directory found: ./cropped_images
Contents: ['Appropriate', 'Inappropriate']
Found 1667 files belonging to 2 classes.
Extracting features from images...
53/53 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step
Feature Map Shape (Batch, Channels): (1667, 2048)
Flattened Feature Shape: (1667, 2048)


In [4]:

# Step 6: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features_flattened, labels, test_size=0.2, random_state=42)

# Step 7: Train SVM classifier
print("Training SVM classifier...")
svm_classifier = svm.SVC(kernel='linear', C=1, random_state=42)  # Linear kernel

svm_classifier.fit(X_train, y_train)
# Step 8: Evaluate the SVM classifier
print("Evaluating SVM classifier...")
y_pred = svm_classifier.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Accuracy Score:", accuracy_score(y_test, y_pred))



# Freeze all layers up to a certain point (e.g., freeze all layers except the top 4 blocks of ResNet50)
for layer in base_model.layers[-94:]:  # Freeze all layers except the last 4 blocks
    layer.trainable = True


print("Training SVM classifier after unfreezing top layer...")

svm_classifier.fit(X_train, y_train)
# Step 8: Evaluate the SVM classifier
print("Evaluating SVM classifier...")
y_pred = svm_classifier.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=class_names))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Step 9: Predict on a new image
def predict_image1(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))  # Add batch dimension
    feature_maps = feature_extractor.predict(img_array)
    flattened_features = feature_maps.flatten().reshape(1, -1)  # Flatten feature maps
    prediction = svm_classifier.predict(flattened_features)
    return class_names[prediction[0]]



Training SVM classifier...
Evaluating SVM classifier...
Classification Report:
               precision    recall  f1-score   support

  Appropriate       0.92      0.93      0.93       169
Inappropriate       0.93      0.92      0.93       165

     accuracy                           0.93       334
    macro avg       0.93      0.93      0.93       334
 weighted avg       0.93      0.93      0.93       334

Accuracy Score: 0.9281437125748503
Training SVM classifier after unfreezing top layer...
Evaluating SVM classifier...
Classification Report:
               precision    recall  f1-score   support

  Appropriate       0.92      0.93      0.93       169
Inappropriate       0.93      0.92      0.93       165

     accuracy                           0.93       334
    macro avg       0.93      0.93      0.93       334
 weighted avg       0.93      0.93      0.93       334

Accuracy Score: 0.9281437125748503


In [ ]:
# Save the feature extractor model
model_save_path = 'Saved models/svm/resnetextractor.keras'  # Specify the file path and name
feature_extractor.save(model_save_path)
print(f"Model saved to {model_save_path}")
import joblib

svm_save_path = 'Saved models/svm'
svm_model_path = os.path.join(svm_save_path, 'svm_model.pkl')
joblib.dump(svm_classifier, svm_model_path)

Model saved to .saved models/svm/resnetextractor.keras


['.saved models/svm\\svm_model.pkl']

In [ ]:
print("ResNet50 Trainable Layers:")
for i, layer in enumerate(base_model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")


In [3]:
import os
import joblib
import tensorflow as tf

# Load the ResNet feature extractor
load_ex = tf.keras.models.load_model('Saved models/svm/resnetextractor.keras')
print("Feature extractor loaded.")

# Load the SVM model
svm_model_path = os.path.join('Saved models/svm', 'svm_model.pkl')
load_svm = joblib.load(svm_model_path)
print("SVM classifier loaded.")

def predict_imageSVM(img_path):
    """
    Predict the class of an image using the saved ResNet-based feature extractor and SVM classifier.

    Args:
        img_path (str): Path to the image file.

    Returns:
        str: Predicted class name.
    """
    # Step 1: Preprocess the image
    img = image.load_img(img_path, target_size=(224, 224))  # Load the image with target size
    img_array = image.img_to_array(img)  # Convert to array
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))  # Preprocess for ResNet50

    # Step 2: Extract features using the loaded feature extractor
    feature_maps = load_ex.predict(img_array)  # Forward pass through feature extractor
    flattened_features = feature_maps.reshape(1, -1)  # Flatten feature maps for SVM input

    # Step 3: Predict the class using the SVM classifier
    prediction = load_svm.predict(flattened_features)  # Predict using SVM

    # Step 4: Convert the predicted class index to a class name
    predicted_class = class_names[prediction[0]]  # Use class_names list to map index to class name

    return predicted_class




Feature extractor loaded.
SVM classifier loaded.


In [ ]:
# Example: Predict the class of an uploaded image
uploaded_image_path = './DARKNET/zeb.jpg'
predicted_class = predict_imageSVM(uploaded_image_path)
print(f"Predicted Class for {new_img_path}: {predicted_class}")

In [12]:
# Test prediction
new_img_path = './DARKNET/TEST.jpg'  # Replace with the path to a new image
predicted_class = predict_image1(new_img_path)
print(f"Predicted Class for {new_img_path}: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Predicted Class for ./DARKNET/TEST.jpg: Appropriate


# CREATING THE CROPPED PERSONS DATASET

In [36]:
def get_yolo_preds_and_save_crops(net, input_img_path, cropped_folder, no_person_folder, confidence_threshold=0.6, overlapping_threshold=0.3, labels=None):
    # Ensure output directories exist
    if not os.path.exists(cropped_folder):
        os.makedirs(cropped_folder)

    if not os.path.exists(no_person_folder):
        os.makedirs(no_person_folder)

    # Generate random colors for each class
    np.random.seed(123)
    colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

    # Get YOLO output layer names
    ln = net.getLayerNames()
    unconnected_out_layers = net.getUnconnectedOutLayers()
    ln = [ln[i - 1] for i in unconnected_out_layers.flatten()]

    # Read the input image
    image = cv2.imread(input_img_path)
    if image is None:
        print(f"Error: Unable to load image {input_img_path}")
        return

    (H, W) = image.shape[:2]

    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(ln)

    # Initialize lists for detections
    boxes = []
    confidences = []
    classIDs = []

    # Loop through each output layer
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            if confidence > confidence_threshold and classID == 0:  # Class ID 0 is "person"
                # Scale bounding boxes back to image dimensions
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # Perform non-maxima suppression to reduce overlapping boxes
    bboxes = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, overlapping_threshold)

    # If no persons detected, save the original image in both folders
    if len(bboxes) == 0:
        # Save the image in both the cropped folder and no_person folder
        no_person_filename = os.path.join(no_person_folder, os.path.basename(input_img_path))
        cv2.imwrite(no_person_filename, image)
        print(f"Saved image with no person detected: {no_person_filename}")

        # Save the entire image in the cropped folder as well
        cropped_filename = os.path.join(cropped_folder, f"no_person_{os.path.basename(input_img_path)}")
        cv2.imwrite(cropped_filename, image)
        print(f"Saved image with no person detected in cropped folder: {cropped_filename}")
        return  # Move on to the next image if no persons are detected

    # Collect and save cropped persons
    for i in bboxes.flatten():
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])

        # Ensure the crop is within the image bounds
        x = max(0, x)
        y = max(0, y)
        w = min(W - x, w)
        h = min(H - y, h)

        # Ensure the cropped region is valid (not empty)
        if w > 0 and h > 0:
            cropped_person = image[y:y + h, x:x + w]

            # Generate a unique filename using the original image's filename, confidence level, and person index
            original_filename = os.path.splitext(os.path.basename(input_img_path))[0]
            cropped_filename = os.path.join(cropped_folder, f"{original_filename}_person_{i + 1}_confidence_{confidences[i]:.2f}.jpg")
            cv2.imwrite(cropped_filename, cropped_person)
            print(f"Saved cropped person: {cropped_filename}")
        else:
            print(f"Warning: Empty crop for {input_img_path} at box {boxes[i]}")


def process_images_in_folder(net, input_folder, output_folder, no_person_folder, confidence_threshold=0.5, overlapping_threshold=0.3, labels=None):
    # List all images in the folder
    for label in ['Appropriate', 'Inappropriate']:  # Iterate over labeled folders
        label_folder = os.path.join(input_folder, label)
        output_label_folder = os.path.join(output_folder, label)

        if not os.path.exists(output_label_folder):
            os.makedirs(output_label_folder)

        # Process each image in the folder
        for img_filename in os.listdir(label_folder):
            img_path = os.path.join(label_folder, img_filename)

            if os.path.isfile(img_path) and img_filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                print(f"Processing image: {img_path}")
                # Get YOLO predictions and save the cropped persons
                get_yolo_preds_and_save_crops(net, img_path, output_label_folder, no_person_folder, confidence_threshold, overlapping_threshold, labels)

try:
    # Read labels from coco.txt
    with open("./DARKNET/coco.txt", "r", encoding="utf-8") as f:
        labels = f.read().strip().split("\n")
except FileNotFoundError:
    print("Error: coco.txt file not found. Please check the file path.")
    exit()

# Load YOLO model
yolo_path = "./DARKNET/model_data/yolov3.cfg"
weights = "./DARKNET/model_data/yolov3.weights"
net = cv2.dnn.readNetFromDarknet(yolo_path, weights)

# Optional: Use CUDA if available
cuda = True
if cuda:
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Define the input folder (your dataset folder) and output folder
input_folder = "./train"  # Folder containing 'Appropriate' and 'Inappropriate'
output_folder = "./cropped_images"  # Folder to save cropped person images
no_person_folder = "./no_persons_detected"  # Folder to save images with no persons detected

# Process all images in the input folder
process_images_in_folder(net, input_folder, output_folder, no_person_folder, confidence_threshold=0.5, overlapping_threshold=0.3, labels=labels)

Processing image: ./train\Appropriate\00000001.jpg
Saved cropped person: ./cropped_images\Appropriate\00000001_person_2_confidence_1.00.jpg
Processing image: ./train\Appropriate\00000002.jpg
Saved cropped person: ./cropped_images\Appropriate\00000002_person_6_confidence_1.00.jpg
Processing image: ./train\Appropriate\0000000296.png
Saved cropped person: ./cropped_images\Appropriate\0000000296_person_5_confidence_1.00.jpg
Processing image: ./train\Appropriate\00000003.jpg
Saved cropped person: ./cropped_images\Appropriate\00000003_person_4_confidence_1.00.jpg
Processing image: ./train\Appropriate\00000004.jpg
Saved cropped person: ./cropped_images\Appropriate\00000004_person_3_confidence_1.00.jpg
Processing image: ./train\Appropriate\00000006.png
Saved cropped person: ./cropped_images\Appropriate\00000006_person_5_confidence_1.00.jpg
Processing image: ./train\Appropriate\00000008.jpg
Saved cropped person: ./cropped_images\Appropriate\00000008_person_3_confidence_1.00.jpg
Processing image

# YOLO + Resnet50&Inceptionv3 Backend

In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception

# Load the classifier model
model_save_path = 'Saved models/resnetxinceptionVVV2.keras'
# model_save_path = 'Saved models/resnetxinceptionV5_TRAINED_ON_CROPPED.keras'
loaded_proposed = tf.keras.models.load_model(model_save_path)
class_names = ['appropriate', 'inappropriate']  # Adjust these based on your model's classes


def predict_image(img_path):
    """
    Predict the class of an image using the combined ResNet50 and InceptionV3 models.
    """
    # Load the image for ResNet50 and InceptionV3
    img_resnet = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224), color_mode='rgb')
    img_inception = tf.keras.preprocessing.image.load_img(img_path, target_size=(299, 299), color_mode='rgb')

    # Preprocess the images
    img_array_resnet = preprocess_resnet(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img_resnet), axis=0))
    img_array_inception = preprocess_inception(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img_inception), axis=0))

    # Predict using the loaded model
    prediction = loaded_proposed.predict([img_array_resnet, img_array_inception])
    predicted_value = prediction[0].item()

    confidence = prediction[0].item()
    interpretation = f"The model is {(1 - confidence) * 100:.2f}% confident the image does not belong to the class '{class_names[1]}'."
    # Return class name based on threshold (0.5)
    print(interpretation)
    return class_names[int(predicted_value > 0.5)]


def get_yolo_preds():
    """
    Run YOLO predictions, save cropped persons, classify them, and label the entire image.
    """
    labels_path = "./DARKNET/coco.txt"
    yolo_cfg = "./DARKNET/model_data/yolov3.cfg"
    yolo_weights = "./DARKNET/model_data/yolov3.weights"
    input_img_path = "./DARKNET/test1.jpg"
    output_img_path = "./DARKNET/output/whole/output.jpg"
    confidence_threshold = 0.5
    overlapping_threshold = 0.5

    # Load COCO labels
    try:
        with open(labels_path, "r", encoding="utf-8") as f:
            labels = f.read().strip().split("\n")
    except FileNotFoundError:
        print(f"Error: {labels_path} not found. Please check the file path.")
        return

    # Initialize YOLO model
    net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)
    if cv2.cuda.getCudaEnabledDeviceCount() > 0:
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    # Read input image
    image = cv2.imread(input_img_path)
    if image is None:
        print(f"Error: Unable to load image {input_img_path}")
        return
    (H, W) = image.shape[:2]

    # Create a blob and perform inference
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward([net.getLayerNames()[i - 1] for i in net.getUnconnectedOutLayers().flatten()])

    # Initialize lists for detections
    boxes, confidences, classIDs = [], [], []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Only keep detections for "person" (class ID 0 in COCO)
            if confidence > confidence_threshold and classID == 0:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # Non-maxima suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, overlapping_threshold)

    image_with_boxes = image.copy()
    inappropriate_detected = False

    # Collect cropped persons and classify them
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y, w, h) = boxes[i]

            # Ensure the cropped region is within bounds
            y_end = min(y + h, image.shape[0])
            x_end = min(x + w, image.shape[1])

            cropped_person = image[y:y_end, x:x_end]

            # Check if the cropped person is valid (non-empty)
            if cropped_person.size == 0:
                print(f"Skipping empty cropped person {i + 1}")
                continue

            cropped_person_path = f"./DARKNET/output/cropped/person_{i + 1}.jpg"

            # Save the cropped person if valid
            if cv2.imwrite(cropped_person_path, cropped_person):
                print(f"Cropped Person {i + 1} Saved: {cropped_person_path}")
            else:
                print(f"Error saving cropped person {i + 1}")

            # Classify the cropped person
            label = predict_image(cropped_person_path)
            print(f"Person {i + 1} classified as: {label}")

            # Check if the person is inappropriate
            if label == 'inappropriate':
                inappropriate_detected = True

            # Draw bounding box on the image with classification
            color = (0, 0, 255) if label == 'inappropriate' else (0, 255, 0)
            cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image_with_boxes, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Save the output image with bounding boxes
    if cv2.imwrite(output_img_path, image_with_boxes):
        print(f"Image with bounding boxes saved to {output_img_path}")

    # Final classification of the image
    if inappropriate_detected:
        print("The entire image is classified as: inappropriate")
    else:
        print("The entire image is classified as: appropriate")


# Run the YOLO predictions
if __name__ == '__main__':
    get_yolo_preds()


Cropped Person 10 Saved: ./DARKNET/output/cropped/person_10.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
The model is 13.97% confident the image does not belong to the class 'inappropriate'.
Person 10 classified as: inappropriate
Skipping empty cropped person 8
Cropped Person 27 Saved: ./DARKNET/output/cropped/person_27.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
The model is 41.84% confident the image does not belong to the class 'inappropriate'.
Person 27 classified as: inappropriate
Cropped Person 22 Saved: ./DARKNET/output/cropped/person_22.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
The model is 29.94% confident the image does not belong to the class 'inappropriate'.
Person 22 classified as: inappropriate
Cropped Person 3 Saved: ./DARKNET/output/cropped/person_3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
The model is 26.43% confident the image does not belong to the class 'inappropriate'.
Person 3 classified as: inappropriate
Cropped Person 16 Saved: ./DARKNET/output/cropped/person_16.jp

# YOLO + Resnet_SVM

In [3]:

import cv2
import numpy as np
import tensorflow as tf
import os
import joblib
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet


class_names = ['appropriate', 'inappropriate']  
# Load the ResNet feature extractor
load_extractor = tf.keras.models.load_model('Saved models/svm/resnetextractor.keras')

# Load the SVM model
svm_model_path = os.path.join('Saved models/svm', 'svm_model.pkl')
load_svm = joblib.load(svm_model_path)


def predict_image(img_path):
    

    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224), color_mode='rgb')
    img_array = preprocess_resnet(np.expand_dims(tf.keras.preprocessing.image.img_to_array(img), axis=0))
   
   
    feature_maps = load_extractor.predict(img_array)  
    flattened_features = feature_maps.reshape(1, -1)  

 
    prediction = load_svm.predict(flattened_features)  
  
    predicted_class = class_names[prediction[0]]  

    return predicted_class


def get_yolo_preds():
    """
    Run YOLO predictions, save cropped persons, classify them, and label the entire image.
    """
    labels_path = "./DARKNET/coco.txt"
    yolo_cfg = "./DARKNET/model_data/yolov3.cfg"
    yolo_weights = "./DARKNET/model_data/yolov3.weights"
    input_img_path = "./DARKNET/test1.jpg"
    output_img_path = "./DARKNET/output/whole/output.jpg"
    confidence_threshold = 0.5
    overlapping_threshold = 0.5

    # Load COCO labels
    try:
        with open(labels_path, "r", encoding="utf-8") as f:
            labels = f.read().strip().split("\n")
    except FileNotFoundError:
        print(f"Error: {labels_path} not found. Please check the file path.")
        return

    # Initialize YOLO model
    net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)
    if cv2.cuda.getCudaEnabledDeviceCount() > 0:
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    # Read input image
    image = cv2.imread(input_img_path)
    if image is None:
        print(f"Error: Unable to load image {input_img_path}")
        return
    (H, W) = image.shape[:2]

    # Create a blob and perform inference
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward([net.getLayerNames()[i - 1] for i in net.getUnconnectedOutLayers().flatten()])

    # Initialize lists for detections
    boxes, confidences, classIDs = [], [], []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Only keep detections for "person" (class ID 0 in COCO)
            if confidence > confidence_threshold and classID == 0:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # Non-maxima suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, overlapping_threshold)

    image_with_boxes = image.copy()
    inappropriate_detected = False

    # Collect cropped persons and classify them
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y, w, h) = boxes[i]

            # Ensure the cropped region is within bounds
            y_end = min(y + h, image.shape[0])
            x_end = min(x + w, image.shape[1])

            cropped_person = image[y:y_end, x:x_end]

            # Check if the cropped person is valid (non-empty)
            if cropped_person.size == 0:
                print(f"Skipping empty cropped person {i + 1}")
                continue

            cropped_person_path = f"./DARKNET/output/cropped/person_{i + 1}.jpg"

            # Save the cropped person if valid
            if cv2.imwrite(cropped_person_path, cropped_person):
                print(f"Cropped Person {i + 1} Saved: {cropped_person_path}")
            else:
                print(f"Error saving cropped person {i + 1}")

            # Classify the cropped person
            label = predict_image(cropped_person_path)
            print(f"Person {i + 1} classified as: {label}")

            # Check if the person is inappropriate
            if label == 'inappropriate':
                inappropriate_detected = True

            # Draw bounding box on the image with classification
            color = (0, 0, 255) if label == 'inappropriate' else (0, 255, 0)
            cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), color, 2)
            cv2.putText(image_with_boxes, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Save the output image with bounding boxes
    if cv2.imwrite(output_img_path, image_with_boxes):
        print(f"Image with bounding boxes saved to {output_img_path}")

    # Final classification of the image
    if inappropriate_detected:
        print("The entire image is classified as: inappropriate")
    else:
        print("The entire image is classified as: appropriate")


# Run the YOLO predictions
if __name__ == '__main__':
    get_yolo_preds()


Cropped Person 10 Saved: ./DARKNET/output/cropped/person_10.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Person 10 classified as: appropriate
Skipping empty cropped person 8
Cropped Person 27 Saved: ./DARKNET/output/cropped/person_27.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Person 27 classified as: appropriate
Cropped Person 22 Saved: ./DARKNET/output/cropped/person_22.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Person 22 classified as: inappropriate
Cropped Person 3 Saved: ./DARKNET/output/cropped/person_3.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
Person 3 classified as: appropriate
Cropped Person 16 Saved: ./DARKNET/output/cropped/person_16.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Person 16 classified as: inappropriate
Cropped Person 13 Saved: ./DARKNET/output/cropped/person_13.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Person 13 classified as: appropriate
Image with bounding boxes saved to ./DARKNET/output/whole/output.jpg
The entire image is classified as: inappropriate
